# Configuración de ambiente y definición de utilidades


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Configuración servidor base de datos transaccional
db_user = 'Estudiante_7'
db_psswd = '6CKNTTSCQR'
source_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'

dest_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/Estudiante_7'

# Driver de conexion
path_jar_driver = '/content/drive/MyDrive/mysql-connector-java-8.0.28.jar'

In [3]:
!pip install pyspark
import os 
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace, mean
from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [5]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

def guardar_db(db_connection_string, df, tabla, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

# Proceso ETL para dimensión Aeropuerto
## Extracción
Se extrae la información requerida para la dimensión Aeropuerto de la tabla aeropuertos

In [6]:
# EXTRACCION
sql_aeropuertos = '''(SELECT sigla, iata, nombre, elevacion, orientacion FROM ProyectoTransaccional.aeropuertos) AS Temp_aeropuertos'''

aeropuertos = obtener_dataframe_de_bd(source_db_connection_string, sql_aeropuertos, db_user, db_psswd)

print(aeropuertos.columns)
aeropuertos.show(5)

['sigla', 'iata', 'nombre', 'elevacion', 'orientacion']
+-----+----+--------------------+---------+-----------+
|sigla|iata|              nombre|elevacion|orientacion|
+-----+----+--------------------+---------+-----------+
|  7fo|    |             la isla|    538.0|           |
|  7fu|    |        la escondida|    564.0|           |
|  7fw|    |           morichito|    720.0|           |
|  7fx|    |carolina del prin...|   6004.0|           |
|  7fy|    |               dubai|     82.0|           |
+-----+----+--------------------+---------+-----------+
only showing top 5 rows



## Transformación
Se verifica el procentaje de información faltante para cada columna. Para lograr esto primero se reemplazan todos los valores vacios por None en el data frame para facilitar los calculos.

In [7]:
# TRANSFORMACION
aeropuertos = aeropuertos.select([when(col(c)=="",None).otherwise(col(c)).alias(c) for c in aeropuertos.columns])
aeropuertos = aeropuertos.select([when(col(c)==0,None).otherwise(col(c)).alias(c) for c in aeropuertos.columns])

Se calcula para cada columna el porcentaje de datos faltantes, en donde se aprecia que para la columna orientacion falta el 100% de los datos y para la columna iata falta el 78% de los datos. Para la columna elevación falta el 0,1% de los datos pero esto se debe a valores iguales a cero (0), por lo que se tomará esta columna como completa. Las columnas sigla y nombre están completas.

In [8]:
aeropuertos.agg(*[(1-(count(c) / count('*'))).alias(c + '_faltante') for c in aeropuertos.columns]).show()

+--------------+------------------+---------------+--------------------+--------------------+
|sigla_faltante|     iata_faltante|nombre_faltante|  elevacion_faltante|orientacion_faltante|
+--------------+------------------+---------------+--------------------+--------------------+
|           0.0|0.7850162866449512|            0.0|0.001628664495114...|                 1.0|
+--------------+------------------+---------------+--------------------+--------------------+



Dado que la columna orientacion no cuenta con ningún dato, y esta columna no es necesaria para la realización de los análisis solicitados por el negocio se procede a eliminar esta columna.

In [9]:
aeropuertos = aeropuertos.drop('orientacion')
aeropuertos.show(5)

+-----+----+--------------------+---------+
|sigla|iata|              nombre|elevacion|
+-----+----+--------------------+---------+
|  7fo|null|             la isla|    538.0|
|  7fu|null|        la escondida|    564.0|
|  7fw|null|           morichito|    720.0|
|  7fx|null|carolina del prin...|   6004.0|
|  7fy|null|               dubai|     82.0|
+-----+----+--------------------+---------+
only showing top 5 rows



Dado que a la columna iata le falta casi el 80% de información, y su información no es categórica ni numérica, ademas de que esta columna no es necesaria para la realización de los análisis solicitados por el negocio, se procede a eliminar esta columna.

In [10]:
aeropuertos = aeropuertos.drop('iata')
aeropuertos.show(5)

+-----+--------------------+---------+
|sigla|              nombre|elevacion|
+-----+--------------------+---------+
|  7fo|             la isla|    538.0|
|  7fu|        la escondida|    564.0|
|  7fw|           morichito|    720.0|
|  7fx|carolina del prin...|   6004.0|
|  7fy|               dubai|     82.0|
+-----+--------------------+---------+
only showing top 5 rows



Para la columna elevacion se ajustan los valores nulos para que vuelvan a tener el valor de cero (0).

In [11]:
means = {}
means['elevacion'] = 0

In [12]:
aeropuertos = aeropuertos.fillna(means)
print('Cantidad de columnas: ', aeropuertos.count())
aeropuertos.show(5)

Cantidad de columnas:  1228
+-----+--------------------+---------+
|sigla|              nombre|elevacion|
+-----+--------------------+---------+
|  7fo|             la isla|    538.0|
|  7fu|        la escondida|    564.0|
|  7fw|           morichito|    720.0|
|  7fx|carolina del prin...|   6004.0|
|  7fy|               dubai|     82.0|
+-----+--------------------+---------+
only showing top 5 rows



Se eliminan las columnas duplicadas.

In [13]:
aeropuertos = aeropuertos.drop_duplicates()
print('Cantidad de columnas: ', aeropuertos.count())
aeropuertos.show(5)

Cantidad de columnas:  299
+-----+--------------+---------+
|sigla|        nombre|elevacion|
+-----+--------------+---------+
|  ypp|german alberto|    735.0|
|  7hs| castillo - ca|     20.0|
|  9ax|    santa cruz|    659.0|
|  a03|        fortul|    730.0|
|  aci| el monasterio|    948.0|
+-----+--------------+---------+
only showing top 5 rows



Se agrega columna de identificación en el DWH.



In [14]:
aeropuertos = aeropuertos.withColumn('idAeropuerto_DWH', f.monotonically_increasing_id() + 1)
aeropuertos.show(5)

+-----+--------------+---------+----------------+
|sigla|        nombre|elevacion|idAeropuerto_DWH|
+-----+--------------+---------+----------------+
|  ypp|german alberto|    735.0|               1|
|  7hs| castillo - ca|     20.0|               2|
|  9ax|    santa cruz|    659.0|               3|
|  a03|        fortul|    730.0|               4|
|  aci| el monasterio|    948.0|               5|
+-----+--------------+---------+----------------+
only showing top 5 rows



## Cargue
Se guarda la información en la tabla Aeropuerto.

In [15]:
# CARGUE
guardar_db(dest_db_connection_string, aeropuertos,'Estudiante_7.Aeropuerto', db_user, db_psswd)

# Proceso ETL para dimensión MiniDimensionAeropuerto
## Extracción
Se extrae la información requerida para la dimensión MiniDimensionAeropuerto de la tabla aeropuertos

In [16]:
# EXTRACCION
sql_mini_dimension_aeropuertos = '''(SELECT longitud_pista, ancho_pista, clase, tipo, numero_vuelos_origen FROM ProyectoTransaccional.aeropuertos) AS Temp_miniDimensionAeropuertos'''

mini_dimension_aeropuertos = obtener_dataframe_de_bd(source_db_connection_string, sql_mini_dimension_aeropuertos, db_user, db_psswd)

print(mini_dimension_aeropuertos.columns)
mini_dimension_aeropuertos.show(5)

['longitud_pista', 'ancho_pista', 'clase', 'tipo', 'numero_vuelos_origen']
+--------------+-----------+-----+-----------+--------------------+
|longitud_pista|ancho_pista|clase|       tipo|numero_vuelos_origen|
+--------------+-----------+-----+-----------+--------------------+
|           800|       10.0|   1A|    Privado|                 2.0|
|          1300|       18.0|   1A|FumigaciÃ³n|                30.0|
|           800|       15.0|   1A|    Privado|                23.0|
|           400|       16.0|   1A|   PÃºblico|                16.0|
|           350|       10.0|   1A|    Privado|                 9.0|
+--------------+-----------+-----+-----------+--------------------+
only showing top 5 rows



## Transformación
Se verifica el procentaje de información faltante para cada columna. Para lograr esto primero se reemplazan todos los valores vacios por None en el data frame para facilitar los calculos.

In [17]:
# TRANSFORMACION
mini_dimension_aeropuertos = mini_dimension_aeropuertos.select([when(col(c)=="",None).otherwise(col(c)).alias(c) for c in mini_dimension_aeropuertos.columns])
mini_dimension_aeropuertos = mini_dimension_aeropuertos.select([when(col(c)==0,None).otherwise(col(c)).alias(c) for c in mini_dimension_aeropuertos.columns])

In [18]:
mini_dimension_aeropuertos.agg(*[(1-(count(c) / count('*'))).alias(c + '_faltante') for c in mini_dimension_aeropuertos.columns]).show()

+-----------------------+--------------------+--------------+-------------+-----------------------------+
|longitud_pista_faltante|ancho_pista_faltante|clase_faltante|tipo_faltante|numero_vuelos_origen_faltante|
+-----------------------+--------------------+--------------+-------------+-----------------------------+
|                    0.0|                 0.0|           0.0|          0.0|                          0.0|
+-----------------------+--------------------+--------------+-------------+-----------------------------+



Se calcula para cada columna el porcentaje de datos faltantes, en donde se aprecia que todas las columnas tienen los valores completos.


Se ajusta valor de numero_vuelos_origen para que no existan valores negativos.

In [19]:
mini_dimension_aeropuertos = mini_dimension_aeropuertos.withColumn('numero_vuelos_origen', when(col('numero_vuelos_origen') < 0, col('numero_vuelos_origen') * -1).otherwise(col('numero_vuelos_origen')))
mini_dimension_aeropuertos.show(5)

+--------------+-----------+-----+-----------+--------------------+
|longitud_pista|ancho_pista|clase|       tipo|numero_vuelos_origen|
+--------------+-----------+-----+-----------+--------------------+
|           800|       10.0|   1A|    Privado|                 2.0|
|          1300|       18.0|   1A|FumigaciÃ³n|                30.0|
|           800|       15.0|   1A|    Privado|                23.0|
|           400|       16.0|   1A|   PÃºblico|                16.0|
|           350|       10.0|   1A|    Privado|                 9.0|
+--------------+-----------+-----+-----------+--------------------+
only showing top 5 rows



Se procede a eliminar los registros duplicados.

In [20]:
mini_dimension_aeropuertos = mini_dimension_aeropuertos.drop_duplicates()
print('Cantidad de columnas: ', mini_dimension_aeropuertos.count())
mini_dimension_aeropuertos.show(5)

Cantidad de columnas:  407
+--------------+-----------+-----+-----------+--------------------+
|longitud_pista|ancho_pista|clase|       tipo|numero_vuelos_origen|
+--------------+-----------+-----+-----------+--------------------+
|           350|       10.0|   UL|    Privado|                11.0|
|           700|       14.0|   1A|    Privado|                10.0|
|          1200|       30.0|   2C|  Aerocivil|              2566.0|
|           600|       10.0|   1A|    Privado|                 1.0|
|          1000|       18.0|   1A|FumigaciÃ³n|                 2.0|
+--------------+-----------+-----+-----------+--------------------+
only showing top 5 rows



Se agrega columna de identificación en el DWH.

In [21]:
mini_dimension_aeropuertos = mini_dimension_aeropuertos.withColumn('idMini_DWH', f.monotonically_increasing_id() + 1)
mini_dimension_aeropuertos.show(5)

+--------------+-----------+-----+-----------+--------------------+----------+
|longitud_pista|ancho_pista|clase|       tipo|numero_vuelos_origen|idMini_DWH|
+--------------+-----------+-----+-----------+--------------------+----------+
|           350|       10.0|   UL|    Privado|                11.0|         1|
|           700|       14.0|   1A|    Privado|                10.0|         2|
|          1200|       30.0|   2C|  Aerocivil|              2566.0|         3|
|           600|       10.0|   1A|    Privado|                 1.0|         4|
|          1000|       18.0|   1A|FumigaciÃ³n|                 2.0|         5|
+--------------+-----------+-----+-----------+--------------------+----------+
only showing top 5 rows



## Cargue
Se guarda la información en la tabla MiniDimensionAeropuerto.

In [22]:
# CARGUE
guardar_db(dest_db_connection_string, mini_dimension_aeropuertos,'Estudiante_7.MiniDimensionAeropuerto', db_user, db_psswd)